Configure matplotlib to use inline mode. The reason for using that is so that when I write code to plot, and then execute it with a simple SHIFT-ENTER, the image will appear directly under that code cell.

In [1]:
%matplotlib inline

In [2]:
# path = "data/dogscats/"
path = "data/dogscats/sample/"

Use division and print functions in the python 3 style (so 1/2 == 0.5, not 0), and things like `print x, sep=', ', end=''` work

In [3]:
from __future__ import division,print_function

Import some other useful things:

In [4]:
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [5]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [6]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

Using Theano backend.


In [7]:
vgg = Vgg16()

In [8]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)

Found 16 images belonging to 2 classes.


In [9]:
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)

Found 8 images belonging to 2 classes.


In [10]:
vgg.finetune(batches)

In [11]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
16/16 [==============================] - 14s - loss: 1.6893 - acc: 0.5625 - val_loss: 0.5039 - val_acc: 0.7500
